PROYECTO INDIVIDUAL DE MLOPs (Machine Learning Operations)

NESTOR CARDONA

ETL

ETL 1RA PARTE
AQUI SE EMPIEZA LA EXTRACCION

PASO 1: SE CARGAN LOS 3 ARCHIVOS JSON Y LOS CONVERTIMOS A DATAFRAMES DE PANDAS
steam_games.json, users_items.json y user_reviews.json

In [154]:
import json
import pandas as pd
import jsonlines
import ast
import gzip
from textblob import TextBlob
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [15]:
'''
AQUI SE CARGA: steam_games.json.gz y se descomprme y luego 
Se pasa a dataset de pandas y se pasa a archivo CSV para luego revisar la data
'''

def descomprime(ruta):
    with gzip.open(ruta,'r') as g:
        for i in g:
            yield json.loads(i)
    return

# Cargar el archivo JSON línea por línea
archivo_json = "steam_games.json.gz"  # Ruta al archivo JSON
rows=[]

for line in descomprime(archivo_json):
     row = pd.json_normalize(line)
     rows.append(row)

games = pd.concat(rows, ignore_index=True)


# Pasar DataFrame games a steam_games_orig.csv
archivo_csv= 'steam_games_orig.csv'
games.to_csv(archivo_csv, index=False)


In [304]:
'''
AQUI SE CARGA: user_reviews.json (descomprimido por winzip)
Se pasa a dataset de pandas, luego se descomprime el campo "reviews"
que es un campo JSON y luego se pasa archivo desanidado a CSV para luego revisar la data
'''

# Cargar el archivo JSON línea por línea
archivo_json = "user_reviews.json"
rows = []

with open(archivo_json, "r",encoding = 'utf-8') as json_file:
    for line in json_file.readlines():
        rows.append(ast.literal_eval(line))

# Convertir los objetos JSON a un DataFrame de pandas
df_users_reviews = pd.DataFrame(rows)

# Crear un bucle para descomprimir el campo 'reviews'
for i, row in df_users_reviews.iterrows():
    review_data = row['reviews']
    new_columns = pd.json_normalize(review_data)
    for col in new_columns.columns:
        df_users_reviews.at[i, col] = new_columns.at[0, col]

# Eliminar la columna original 'reviews' si es necesario
df_users_reviews.drop('reviews', axis=1, inplace=True)

df_users_reviews.rename(columns={'item_id': 'id'}, inplace=True)

# Pasar DataFrame users_reviews a users_reviews_orig.csv
archivo_csv = 'users_reviews_orig.csv'
df_users_reviews.to_csv(archivo_csv, index=False)

In [ ]:
'''
AQUI SE CARGA: users_items.json (descomprimido por winzip)
Se pasa a dataset de pandas, luego se descomprime el campo "items"
que es un campo JSON y luego se pasa archivo desanidado a CSV para luego revisar la data
'''

# Cargar el archivo JSON línea por línea
archivo_json = "users_items.json"
rows = []

with open(archivo_json, "r",encoding = 'utf-8') as json_file:
    for line in json_file.readlines():
        rows.append(ast.literal_eval(line))
# Convertir los objetos JSON a un DataFrame de pandas
df_user_items = pd.DataFrame(rows)

# Crear un bucle para descomprimir el campo 'reviews'
for i, row in df_user_items.iterrows():
    review_data = row['items']
    new_columns = pd.json_normalize(review_data)
    for col in new_columns.columns:
        df_user_items.at[i, col] = new_columns.at[0, col]

# Eliminar la columna original 'reviews' si es necesario
df_user_items.drop('items', axis=1, inplace=True)


# Pasar DataFrame users_reviews a users_reviews_orig.csv
archivo_csv = 'users_items_orig.csv'
df_user_items.to_csv(archivo_csv, index=False)



ETL 2DA PARTE
AQUI SE EMPIEZA A TRANSFORMAR LA INFORMACION EXTRAIDA

PASO 2: SE DEPURAN LOS 3 ARCHIVOS CSV EXTRAIDOS EN LA 1RA PARTE
steam_games_orig.csv, users_items_orig.csv y user_reviews_orig.csv

# Cargar el archivo users_reviews_orig.csv

BORRAMOS LOS NaN DEL CAMPO 'item_id' DE df_users_reviews (users_items_orig.csv)

# DEPURACION DE DATOS EN user_reviews

In [229]:
# Cargar el archivo CSV
df_users_reviews = pd.read_csv('users_reviews_orig.csv')

# Eliminar filas con valores NaN en el campo 'item_id'
df_users_reviews = df_users_reviews.dropna(subset=['item_id'])


# Contar cuántos valores NaN hay en una columna específica
nan_count_column = df_users_reviews['user_id'].isna().sum()
print("Número de NaN en la columna 'user_id':", nan_count_column)

# Contar cuántos valores NaN hay en todas las columnas del DataFrame
nan_count_dataframe = df_users_reviews.isna().sum()
print("Número de NaN en cada columna del DataFrame:")
print(nan_count_dataframe)

Número de NaN en la columna 'user_id': 0
Número de NaN en cada columna del DataFrame:
user_id            0
user_url           0
funny          21567
posted             0
last_edited    23744
item_id            0
helpful            0
recommend          0
review            12
dtype: int64


BORRAMOS LOS NaN DEL CAMPO 'item_id' DE df_user_items (users_items_orig.csv)

# DEPURACION DE DATOS EN user_items

In [231]:
# Cargar el archivo CSV
df_user_items = pd.read_csv('users_items_orig.csv')

# Eliminar filas con valores NaN en el campo 'item_id'
df_user_items = df_user_items.dropna(subset=['item_id'])

# Contar cuántos valores NaN hay en una columna específica
nan_count_column = df_user_items['steam_id'].isna().sum()
print("Número de NaN en la columna 'steam_id':", nan_count_column)

# Contar cuántos valores NaN hay en todas las columnas del DataFrame
nan_count_dataframe = df_user_items.isna().sum()

print("Número de NaN en cada columna del DataFrame:")
print(nan_count_dataframe)


# Pasar DataFrame users_reviews a users_reviews_orig.csv
archivo_csv = 'users_items_sin_NaN.csv'
df_user_items.to_csv(archivo_csv, index=False)

Número de NaN en la columna 'steam_id': 0
Número de NaN en cada columna del DataFrame:
user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64


TRANSFORMACIONES EN steam_games 

CREAR CAMPO 'year' SACADO DEL CAMPO 'release_date'
CREAR CAMPOS CON LOS GENEROS MAS USADOS (99.3%), SACADOS DEL CAMPO 'tags' 
ELIMINAR COLUMNAS NO NECESARIAS PARA LOS ENDPOINTS

In [ ]:
# Cargar el archivo CSV
df_steam_games = pd.read_csv('steam_games_orig.csv')

#Agregar campo 'year' del sacado del campo 'release_date'
df_steam_games['year'] = pd.to_datetime(df_steam_games['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if pd.notnull(x) else None)


# Eliminar filas con valores NaN en el campo 'id'
df_steam_games = df_steam_games.dropna(subset=['id'])

# Eliminar filas con valores NaN en el campo 'tags' que se va a usar para 
# GENERO ya que el campo 'genre' tiene muchos mas NaN y la info en ambas columnas es igual
df_steam_games = df_steam_games.dropna(subset=['tags'])


# Contar cuántos valores NaN hay en una columna específica
nan_count_column = df_steam_games['id'].isna().sum()
print("Número de NaN en la columna 'id':", nan_count_column)

# Contar cuántos valores NaN hay en todas las columnas del DataFrame
nan_count_dataframe = df_steam_games.isna().sum()
print("Número de NaN en cada columna del DataFrame:")
print(nan_count_dataframe)

In [309]:
# Obtener el número de filas
num_filas = df_steam_games.shape[0]

# Imprimir el número de filas
print(f"El DataFrame tiene {num_filas} filas.")

El DataFrame tiene 31971 filas.


# SE CRE EL DF df_games_tags CON LOS GENEROS DE LA COLUMNA 'tags' 
# LUEGO ORDENO DESCENCENTEMENTE LOS MAS IMPORTANTES Y CON LOS MAS RELEVANTES
# CREO EL DF tags_pareto PARA CONCATENARLO AL FINAL AL DF  df_steam_games


In [234]:
# Se crea un DF con los generos de la columna 'tags' 

# Verificar si el valor es una lista antes de usar ast.literal_eval
df_steam_games['tags'] = df_steam_games['tags'].apply(lambda x: x if isinstance(x, list) else ast.literal_eval(x))
#df_steam_games['tags'] = df_steam_games['tags'].apply(lambda x: ast.literal_eval(x))

# Luego, continúa con el procesamiento
games_tags = pd.DataFrame(df_steam_games['tags'].tolist())
games_tags_obj = games_tags.stack()
df_games_tags = pd.get_dummies(games_tags_obj)
df_games_tags=df_games_tags.groupby(level=[0], axis=0).sum()


# Ordeno en orden descendente los 15 GENEROS mas importantes, con mas seleccion

In [216]:
# Ordeno en orden descendente los 15 GENEROS mas importantes, con mas seleccion
df_games_tags.sum().sort_values(ascending=False).head(15)/len(df_games_tags)

Indie               0.551124
Action              0.406744
Adventure           0.307466
Casual              0.304620
Simulation          0.241782
Strategy            0.236652
RPG                 0.185137
Singleplayer        0.135904
Free to Play        0.075256
Multiplayer         0.074536
Great Soundtrack    0.069907
Puzzle              0.066029
Early Access        0.060899
2D                  0.060836
Atmospheric         0.060492
dtype: float64

In [235]:
# Selecciono los GENEROS mas importantes del DF para concatenarlos 

tags_pareto=  df_games_tags[['Indie','Action','Adventure','Casual','Simulation','Strategy','RPG','Singleplayer','Free to Play','Multiplayer']]

In [310]:
# Obtener el número de filas
num_filas = tags_pareto.shape[0]

# Imprimir el número de filas
print(f"El DataFrame tiene {num_filas} filas.")

El DataFrame tiene 31971 filas.


SE CONCATENA AL FINAL DEL DF df_games_final EL DF tags_pareto


In [ ]:
# Restablece el índice de df_steam_games antes de la concatenación
df_steam_games.reset_index(drop=True, inplace=True)

# Concatena los DataFrames sin duplicar filas
df_games_final = pd.concat([df_steam_games, tags_pareto], axis=1)
df_games_final.head()

In [312]:
# Obtener el número de filas
num_filas = df_games_final.shape[0]

# Imprimir el número de filas
print(f"El DataFrame tiene {num_filas} filas.")

El DataFrame tiene 31971 filas.


# BORRAMOS LAS COLUMNAS NO NECESARIAS PARA LOS ENDPOINTS PARA QUE SEAN RAPIDAS LAS QUERYS
 'publisher','genres','title','url','release_date','tags','reviews_url','discount_price','specs','price','early_access','metascore','developer','items','items_count'

In [240]:
# BORRAMOS LAS COLUMNAS NO NECESARIAS PARA LOS ENDPOINTS PARA QUE SEAN RAPIDAS LAS QUERYS
df_games_final = df_games_final.drop(['publisher','genres','title','url','release_date','tags','reviews_url','discount_price','specs','price','early_access','metascore','developer','items','items_count'], axis=1)

In [241]:
df_games_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31971 entries, 0 to 31970
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   app_name      31970 non-null  object 
 1   id            31971 non-null  float64
 2   user_id       0 non-null      object 
 3   steam_id      0 non-null      float64
 4   year          29621 non-null  object 
 5   Indie         31971 non-null  int64  
 6   Action        31971 non-null  int64  
 7   Adventure     31971 non-null  int64  
 8   Casual        31971 non-null  int64  
 9   Simulation    31971 non-null  int64  
 10  Strategy      31971 non-null  int64  
 11  RPG           31971 non-null  int64  
 12  Singleplayer  31971 non-null  int64  
 13  Free to Play  31971 non-null  int64  
 14  Multiplayer   31971 non-null  int64  
dtypes: float64(2), int64(10), object(3)
memory usage: 3.7+ MB


SE CREA EL ARCHIVO CSV FINAL DE GAMES games_final.csv PARA ENDPOINTS

In [242]:
# Pasar DataFrame df_games_final a games_final.csv listo para los ENDPOINTS
archivo_csv = 'games_final.csv'
df_games_final.to_csv(archivo_csv, index=False)

TRANSFORMACIONES EN user_reviews 

CREAR CAMPO 'year' SACADO DEL CAMPO 'posted'
CREAR CAMPO 'sentimiento' CON LA FUNCION analizar_sentimiento SACADO DEL CAMPO 'review'
ELIMINAR COLUMNAS NO NECESARIAS PARA LOS ENDPOINTS

In [ ]:

df_reviews_final = pd.read_csv('users_reviews_orig.csv')
df_reviews_final.info()

# SE CREA CAMPO 'sentimiento' CON ANALISIS DE SENTIMIENTO AL CAMPO 'review'

In [344]:
def analizar_sentimiento(texto):
    if isinstance(texto, str):  # Verificar si es una cadena de caracteres
        analysis = TextBlob(texto)
        # Clasificar el sentimiento como positivo, negativo o neutral
        if analysis.sentiment.polarity > 0:
            return 'Positivo'
        elif analysis.sentiment.polarity < 0:
            return 'Negativo'
    return 'Neutral'

# Aplicar el análisis de sentimiento al campo 'recommended' y crear un nuevo campo 'sentimiento'
df_reviews_final['sentimiento'] = df_reviews_final['review'].apply(analizar_sentimiento)

# Verificar el resultado
print(df_reviews_final[['review', 'sentimiento']])


                                                  review sentimiento
0      Simple yet with great replayability. In my opi...    Positivo
1      I know what you think when you see this title ...    Positivo
2      A suitably punishing roguelike platformer.  Wi...    Positivo
3      This game... is so fun. The fight sequences ha...    Positivo
4                                                Git gud     Neutral
...                                                  ...         ...
25794  I cried in the end its so sadding ]'; I wish l...    Positivo
25795  Gra naprawdę fajna.Ale jest kilka rzeczy do kt...     Neutral
25796                                          Well Done     Neutral
25797  this is a very fun and nice 80s themed shooter...    Positivo
25798  had so much fun plaing this and collecting res...    Positivo

[25799 rows x 2 columns]


In [345]:
# Pasar DataFrame df_games_final a games_final.csv listo 
archivo_csv = 'reviews_sentim.csv'
df_reviews_final.to_csv(archivo_csv, index=False)

# SE CREA CAMPO 'year' EXTRAIDO DEL CAMPO 'posted'

In [ ]:
# Función para extraer el año de la cadena 'posted'
def extract_year(posted_str):
    if isinstance(posted_str, str):
        match = re.search(r'\d{4}', posted_str)
        if match:
            return match.group()
    return None  # Opcionalmente, puedes asignar un valor predeterminado en lugar de None
df_reviews_final = pd.read_csv('reviews_sentim.csv')

# Aplicar la función extract_year y crear la columna 'year'
df_reviews_final['year'] = df_reviews_final['posted'].apply(extract_year)

# Mostrar el DataFrame resultante
print(df_reviews_final)


# BORRAMOS CAMPOS INNECESARIOS DE df_reviews_final PARA DAR PASO AL DEFINITIVO
# QUE SE USARA PARA LOS ENDPOINTS 

In [348]:
# Cambiamos el typo float del campo a object
df_reviews_final['id'] = df_reviews_final['id'].astype(object)


# BORRAMOS LAS COLUMNAS NO NECESARIAS PARA LOS ENDPOINTS PARA QUE SEAN RAPIDAS LAS QUERYS
df_reviews_final = df_reviews_final.drop(['user_url','funny','posted','last_edited','helpful'], axis=1)

In [349]:
df_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      25799 non-null  object
 1   id           25771 non-null  object
 2   recommend    25771 non-null  object
 3   review       25759 non-null  object
 4   sentimiento  25799 non-null  object
 5   year         21069 non-null  object
dtypes: object(6)
memory usage: 1.2+ MB


In [350]:
# Renombrar campo
#df_reviews_final.rename(columns={'item_id': 'id'}, inplace=True)

# Cambiamos el typo float del campo a object
df_reviews_final['id'] = df_reviews_final['id'].astype(object)

# Pasar DataFrame df_reviews_final a reviews_final.csv listo para los ENDPOINTS
archivo_csv = 'reviews_final.csv'
df_reviews_final.to_csv(archivo_csv, index=False)

TRANSFORMACIONES EN user_items 

RENOMBRAR CAMPO 'item_id' A 'id' PARA MEJORAR JOINS 
ELIMINAR COLUMNAS NO NECESARIAS PARA LOS ENDPOINTS
CREAR ARCHIVO FINAL CSV 'items_final.csv'


In [351]:
df_items_final = pd.read_csv('users_items_orig.csv')
df_items_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           88310 non-null  object 
 1   items_count       88310 non-null  int64  
 2   steam_id          88310 non-null  int64  
 3   user_url          88310 non-null  object 
 4   item_id           71504 non-null  float64
 5   item_name         71504 non-null  object 
 6   playtime_forever  71504 non-null  float64
 7   playtime_2weeks   71504 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 5.4+ MB


In [352]:
# BORRAMOS LAS COLUMNAS NO NECESARIAS PARA LOS ENDPOINTS PARA QUE SEAN RAPIDAS LAS QUERYS
df_items_final = df_items_final.drop(['items_count','user_url','playtime_2weeks','steam_id','item_name'], axis=1)

# Renombrar la columna 'old_name' a 'new_name'
df_items_final.rename(columns={'item_id': 'id'}, inplace=True)

# Cambiamos el typo float del campo a object
df_items_final['id'] = df_items_final['id'].astype(object)

# Pasar DataFrame df_items_final a reviews_final.csv listo para los ENDPOINTS
archivo_csv = 'items_final.csv'
df_items_final.to_csv(archivo_csv, index=False)

In [417]:

games = pd.read_csv('games_final.csv')
reviews = pd.read_csv('reviews_final.csv')
items = pd.read_csv('items_final.csv')

items_horas = pd.read_csv('itemsPorHorasJugadas.csv')
Jugad_año = pd.read_csv('UserJugadoresPorYear.csv')


# MACHINE LEARNING
MACHINE LEARNIG

SE ELIMINAN REVIEWS VACIOS Y DE OTROS IDIOMAS DIFERENTES A INGLES

SE CREA EL ARCHIVO reviews_ML.csv PARA EL ENDPOINT DE MACHINE LEARNING

In [4]:
# Cargar el dataframe 'reviews' desde tu fuente de datos
df = pd.read_csv('reviews_final.csv')  # Reemplaza 'reviews.csv' con tu fuente de datos
# Asumiendo que 'id' es la columna que contiene el ID del producto y 'review' es la columna de texto
# Si es necesario, también puedes preprocesar y limpiar los datos aquí

# Reemplazar valores NaN en la columna "review" con cadenas vacías y luego eliminar esas filas
df['review'].fillna('', inplace=True)
df.drop(df[df['review'] == ''].index, inplace=True)

##############

# Filtra las filas con texto en inglés
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return None

# Añade una nueva columna llamada 'language' que contendrá el idioma detectado
df['language'] = df['review'].apply(lambda x: detect_language(x) if pd.notna(x) else None)

# Filtra las filas que están en inglés (código 'en' para inglés)
df = df[df['language'] == 'en']

# Elimina la columna 'language' si ya no la necesitas
df.drop(columns=['language'], inplace=True)


#################

df_games=pd.read_csv('games_final.csv')

df_games['id'] = df_games['id'].astype(str)
df['id'] = df['id'].astype(str)

# Realizar la fusión de los DataFrames usando el campo 'id' como clave de unión
df = df.merge(df_games[['id', 'app_name']], on='id', how='inner')

# Renombrar la columna 'app_name' a 'name'
df.rename(columns={'app_name': 'name'}, inplace=True)

# Ahora, el DataFrame 'df' contiene el campo 'name' que se extrajo de 'df_games'

df.to_csv('reviews_ML.csv')
